## IQ Seminar - Nov 07, 2015
 
Master of Financial Engineering Program, Baruch College   
  
<img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2014/09/BCCUNYstacked_BLK.jpg" align = "left" width=160>  

* Weiyi Chen, weiyi.chen@baruchmail.cuny.edu

Today's questions all from Dan, from recent interviews with GS/MS/JPM etc.

# Definition Questions 

## 1. Hash Table

### 1.1 What is hash tabe?

#### Associative array

A hash table is a data structure used to implement an associative array that can map keys to values. A hash table uses a hash function to compute an index into an array of buckets, from which the desired value can be found.

#### Collisions

The hash function will assign each key to a unique bucket, but it is possible that two keys will generate an identical hash causing both keys to point to the same bucket. Instead, most hash table designs assume that hash collisions—different keys that are assigned by the hash function to the same bucket—will occur and must be accommodated in some way. (Chaining, open addressing)

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/b/bf/Hash_table_5_0_1_1_1_1_0_SP.svg/380px-Hash_table_5_0_1_1_1_1_0_SP.svg.png'>

#### Average cost / complexity

In a well-dimensioned hash table, the average cost for each lookup is independent of the number of elements stored in the table. Many hash table designs also allow arbitrary insertions and deletions of key-value pairs, at constant average cost per operation.

### 1.2 How to implement it?

In [1]:
class HashTable:
    def __init__(self):
        self.size = 11
        self.slots = [None] * self.size
        self.data = [None] * self.size
        
    def put(self,key,data):
      hashvalue = self.hashfunction(key,len(self.slots))

      if self.slots[hashvalue] == None:
        self.slots[hashvalue] = key
        self.data[hashvalue] = data
      else:
        if self.slots[hashvalue] == key:
          self.data[hashvalue] = data  #replace
        else:
          nextslot = self.rehash(hashvalue,len(self.slots))
          while self.slots[nextslot] != None and \
                          self.slots[nextslot] != key:
            nextslot = self.rehash(nextslot,len(self.slots))

          if self.slots[nextslot] == None:
            self.slots[nextslot]=key
            self.data[nextslot]=data
          else:
            self.data[nextslot] = data #replace

    def hashfunction(self,key,size):
         return key%size

    def rehash(self,oldhash,size):
        return (oldhash+1)%size
    
    def get(self,key):
      startslot = self.hashfunction(key,len(self.slots))

      data = None
      stop = False
      found = False
      position = startslot
      while self.slots[position] != None and  \
                           not found and not stop:
         if self.slots[position] == key:
           found = True
           data = self.data[position]
         else:
           position=self.rehash(position,len(self.slots))
           if position == startslot:
               stop = True
      return data

    def __getitem__(self,key):
        return self.get(key)

    def __setitem__(self,key,data):
        self.put(key,data)

In [2]:
H=HashTable()
H[54]="cat"
H[26]="dog"
H[93]="lion"
H[17]="tiger"
H[77]="bird"
H[31]="cow"
H[44]="goat"
H[55]="pig"
H[20]="chicken"
H.slots

[77, 44, 55, 20, 26, 93, 17, None, None, 31, 54]

In [3]:
H.data

['bird',
 'goat',
 'pig',
 'chicken',
 'dog',
 'lion',
 'tiger',
 None,
 None,
 'cow',
 'cat']

## 2. What design pattern do you know?

- Design patterns are optimized, reusable solutions to the programming problems that we encounter every day. 
- A design pattern is not a class or a library that we can simply plug into our system; it's much more than that. It is a template that has to be implemented in the correct situation.

### 2.1 Singleton

Singleton pattern provides a mechanism to limit the number of the instances of the class to one. Thus the same object is always shared by different parts of the code.

<img src='http://www.noesispoint.com/img/DesignPattern/Singleton-Design-Pattern.jpg'>

In [8]:
class Singleton(Exception):
    '''
    Singleton exception(error) class

    Because Python has no private constructors we have to find some other way to prevent instantiations. 
    Our approach is to raise an exception if the Singleton object is already instantiated 
    (private class attribute __single is other than None). The exception object is the Singleton instance
    '''
    __single = None
    def __init__( self ):
        if Singleton.__single:
            raise Singleton.__single
        Singleton.__single = self 

# Creation
        
def Handle( x = Singleton ):

    try:
        single = x()
    except Singleton as s:
        single = s
    return single 

# Subclasses

'''
class Singleton has subclasses Child and Junior as shown. Because there exists an is-a relationship between 
base class and a subclass there can be only one instance of the whole hierarchy. Singleton or Junior instance 
can be created with Handle function. The constructor of the Child class takes one - explicit - argument so the 
instantiation must be done with the direct constructor call or a specialized handle function. 
'''

class Child( Singleton ):
    def __init__( self, name ):
        Singleton.__init__( self )
        self.__name = name
    def name( self ):
        return self.__name

class Junior( Singleton ):
    pass

In [9]:
child = Child( 'Monty' )
junior = Handle( Junior )
junior.name()

'Monty'

In [10]:
single = Handle()
single

__main__.Child('Monty')

In [11]:
child = Child('ABC') # Error

Child: Monty

### 2.2 Chain of Responsibility

- The chain of Responsibility pattern is to create a system that can serve different requests in a hierarchical manner. 

- In other words if an object that is a part of the system does not know how to respond to the given request it passes it along the object tree. Each object along the route of the request can take the responsibility and serve the request.

<img src='http://i.ytimg.com/vi/BsuhLJb6vo0/maxresdefault.jpg'>

In [12]:
class Event:
    def __init__( self, name ):
        self.name = name

class Widget:
    '''
    The idea behind the Command Dispatch pattern is to check at runtime whether the object has a proper method or not.
    '''
    def __init__( self, parent = None ):
        self.__parent = parent
    def Handle( self, event ):
        handler = 'Handle_' + event.name
        if hasattr( self, handler ):
            method = getattr( self, handler )
            method( event )
        elif self.__parent:
            self.__parent.Handle( event )
        elif hasattr( self, 'HandleDefault' ):
            self.HandleDefault( event )    

When an event object is passed to Handle method, one of the four things may happen:

- Each event has an associated name. If widget has a corresponding method (named Handle_eventname ), it is executed
- If the object is not the last one in the chain, it passes the event object to its parent
- If the object has a default handler for all kinds of events (HandleDefault method) it is invoked if the object is the last possible handler. Because HandleDefault should always be available only on the end of the chain, it is not necessary or even sensible to have it in the interface of the base class.
- The object dismisses the event if it can neither handle or forward it.

In [14]:
class MainWindow( Widget ):
    def Handle_close( self, event ):
        print('MainWindow: ' + event.name)
    def HandleDefault( self, event ):
        print('Default: ' + event.name)
        
class SendDialog( Widget ):
    def Handle_paint( self, event ):
        print('SendDialog: ' + event.name)

class MsgText( Widget ):
    def Handle_down( self, event ):
        print('MsgText: ' + event.name)

In [15]:
# Define hierarchi

mw = MainWindow()
sd = SendDialog( mw )
msg = MsgText( sd )

# Define event
edown = Event( 'down' )
epaint = Event( 'paint' )
eodd = Event( 'odd' )

In [16]:
msg.Handle( edown )

MsgText: down


In [17]:
msg.Handle( epaint )

SendDialog: paint


In [18]:
msg.Handle( eodd )

Default: odd


#### A python trick

In [20]:
def PrintName( e ):
    print ('Name: ' + e.name)
    
sd.Handle_odd = PrintName
msg.Handle( eodd )

Name: odd


In [21]:
del sd.Handle_odd
msg.Handle( eodd )

Default: odd


### 2.3 Proxy

- The Proxy pattern is used when an object has to be shielded from its clients. 

- There may be a number of reasons for this: reference counting, different levels of access rights, lazy evaluation of the state of the object and so on.

- The client does not need to know that it is not accessing the real object (Subject) directly. In other words the proxy substitutes for the real thing.

<img src='https://sourcemaking.com/files/v2/content/patterns/Proxy_example1-2x.png'>

In [68]:
# Real subject

class RGB:
    def __init__( self, red, green, blue ):
        self.__red = red
        self.__green = green
        self.__blue = blue
    def Red( self ):
        return self.__red
    def Green( self ):
        return self.__green
    def Blue( self ):
        return self.__blue 


In [74]:
# Proxy base class, return the same subject itself

class Proxy:
    def __init__( self, subject ):
        self.__subject = subject
    def __getattr__( self, name ):
        return getattr( self.__subject, name )
    def html_str(self):
        return 'rgb(%i,%i,%i)'%(self.Red(), self.Green(), self.Blue())
    
# Hacker

class NoBlueProxy( Proxy ):
    def Blue( self ):
        return 0

In [75]:
from IPython.display import HTML
rgb = RGB( 100, 192, 240 )

In [76]:
proxy = Proxy( rgb )
print(proxy.html_str())
HTML(cl.to_html({'RGB':[proxy.html_str()]}))

rgb(100,192,240)


In [78]:
noblue = NoBlueProxy( rgb )
print(noblue.html_str())
HTML(cl.to_html({'RGB':[noblue.html_str()]}))

rgb(100,192,0)


### 2.4 Bridge Pattern

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/c/cf/Bridge_UML_class_diagram.svg/500px-Bridge_UML_class_diagram.svg.png'>

In [99]:
class AbstractInterface:
    """ Target interface.
    This is the target interface, that clients use.
    """
    def someFunctionality(self):
        raise NotImplemented()


class Bridge(AbstractInterface):
    """ Bridge class.
    
    This class forms a bridge between the target
    interface and background implementation.
    """
    def __init__(self):
        self.__implementation = None


class UseCase1(Bridge):
    """ Variant of the target interface.
    This is a variant of the target Abstract interface.
    It can do something little differently and it can
    also use various background implementations through
    the bridge.
    """
    def __init__(self, implementation):
        self.__implementation = implementation
    def someFunctionality(self):
        print ("UseCase1: ",)
        self.__implementation.anotherFunctionality()

class UseCase2(Bridge):
    def __init__(self, implementation):
        self.__implementation = implementation
    def someFunctionality(self):
        print ("UseCase2: ",)
        self.__implementation.anotherFunctionality()

class ImplementationInterface:
    """ Interface for the background implementation.
    This class defines how the Bridge communicates
    with various background implementations.
    """
    def anotherFunctionality(self):
        raise NotImplemented

class Linux(ImplementationInterface):
    """ Concrete background implementation.
    A variant of background implementation, in this
    case for Linux!
    """
    def anotherFunctionality(self):
        print ("Linux!")

class Windows(ImplementationInterface):
    def anotherFunctionality(self):
        print ("Windows.")

def main():
    linux = Linux()
    windows = Windows()

    # Couple of variants under a couple
    # of operating systems.
    useCase = UseCase1(linux)
    useCase.someFunctionality()

    useCase = UseCase1(windows)
    useCase.someFunctionality()

    useCase = UseCase2(linux)
    useCase.someFunctionality()

    useCase = UseCase2(windows)
    useCase.someFunctionality()

if __name__ == "__main__":
    main()

UseCase1: 
Linux!
UseCase1: 
Windows.
UseCase2: 
Linux!
UseCase2: 
Windows.


## 3. Principles of OOP

### 3.1 Encapsulation

Encapsulation is an Object Oriented Programming concept that binds together the data and functions that manipulate the data, and that keeps both safe from outside interference and misuse.

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/CPT-OOP-interfaces.svg/2000px-CPT-OOP-interfaces.svg.png', width=600>

### 3.2 Inheritance

Inheritance (OOP) is when an object or class is based on another object (prototypal inheritance) or class (class-based inheritance), using the same implementation specifying implementation to maintain the same behavior (realizing an interface; inheriting behavior).


### 3.3 Polymorphism

Polymorphism refers to a programming language's ability to process objects differently depending on their data type or class. More specifically, it is the ability to redefine methods for derived classes.

<img src='http://www3.ntu.edu.sg/home/ehchua/programming/java/images/OOP_PersonStudnetTeacher.png'>

## 4. Virtual Function

### 4.1 What's virtual function

- A virtual function (virtual method) is a function whose behavior can be overridden within an inheriting class by a function with the same signature. 

- This concept is an important part of the polymorphism portion of object-oriented programming (OOP).

### 4.2 Why do we need it

In object-oriented programming, when a derived class inherits from a base class, an object of the derived class may be referred to via a pointer or reference of the base class type instead of the derived class type. If there are base class methods overridden by the derived class, the method actually called by such a reference or pointer can be bound either 'early' (by the compiler), according to the declared type of the pointer or reference, or 'late' (i.e., by the runtime system of the language), according to the actual type of the object referred to.

<img src='http://www.studyvilla.com/Images/poly.jpg'>

```cpp
class Animal {
    void /*non-virtual*/ move(void) { 
        std::cout << "This animal moves in some way" << std::endl; 
    }
    virtual void eat(void) {}
};

// The class "Animal" may possess a definition for eat() if desired.
class Wolf : public Animal {
    // The non virtual function move() is inherited but not overridden
    void eat(void) { 
        std::cout << "Llamas eat grass!" << std::endl; 
    }
};
```

### 4.3 Abstract Class (ABC) / Pure Virtual

A pure virtual function or pure virtual method is a virtual function that is required to be implemented by a derived class if the derived class is not abstract. Classes containing pure virtual methods are termed "abstract" and $\textbf{they cannot be instantiated directly}$. A subclass of an abstract class can only be instantiated directly if $\textbf{all inherited pure virtual methods have been implemented}$ by that class or a parent class. Pure virtual methods typically have a declaration (signature) and no definition (implementation).

Although pure virtual methods typically have no implementation in the class that declares them, pure virtual methods in C++ are permitted to contain an implementation in their declaring class, providing default behaviour that a derived class can delegate to, if appropriate.

### 4.4 Virtual Destructor

- Object-oriented languages typically manage memory allocation and de-allocation automatically when objects are created and destroyed. However, some object-oriented languages allow a custom destructor method to be implemented, if desired. (Python, Java)

- If the language in question uses automatic memory management, the custom destructor (generally called a finalizer in this context) that is called is certain to be the appropriate one for the object in question. For example, if an object of type Wolf that inherits Animal is created, and both have custom destructors, the one called will be the one declared in Wolf.

In [12]:
class FooType(object):
    def __init__(self, id):
        self.id = id
        print (self.id, 'born')

    def __del__(self):
        print (self.id, 'died')

def test():
    ft3 = FooType(3)
    
test()

3 born
3 died


In [97]:
class Base(object):
    def __init__(self, id):
        self.id = id
        print (self.id, 'base born')

    def __del__(self):
        print (self.id, 'base died')
        
class Derived(Base):
    def __del__(self):
        print (self.id, 'derived died')

def test2():
    derived = Derived(1)
    
test2()

1 base born
1 derived died


- In manual memory management contexts, the situation can be more complex. If an object of type Wolf is created but pointed to by an Animal pointer, and it is this Animal pointer type that is deleted, the destructor called may actually be the one defined for Animal and not the one for Wolf, unless the destructor is virtual. This is particularly the case with C++, where the behaviour is a common source of programming errors.

```cpp
int main(int argc, char const *argv[])
{
    Animal* AnimalPtr = new Wolf;
    
    delete AnimalPtr; 
    // if the destructor in Animal class is not virtual, then Animal destructor is called, which might be wrong.
    // if the destructor in Animal class is virtual, then Wold destructor is called.
    
	return 0;
}

```

In [20]:
d = {}

d = dict((i, i*10) for i in range(10))

print(d)
for key, value in d.iteritems():
    print(key, value)

{0: 0, 1: 10, 2: 20, 3: 30, 4: 40, 5: 50, 6: 60, 7: 70, 8: 80, 9: 90}


AttributeError: 'dict' object has no attribute 'iteritems'

## 5. Binary Search Tree

### 5.1 Difference between hash table and binary search tree

- A hash table can insert and retrieve elements in $O(1)$. A binary search tree can insert and retrieve elements in $O(log(n))$, which is quite a bit slower than the hash table which can do it in $O(1)$.

- A hash table is an unordered data structure, while a binary search tree is a sorted data structure.

### 5.2 Examples

- When designing a cell phone, you want to keep as much data as possible available for data storage, use hash table
    
- Because a binary search tree is already sorted, there will be no need to waste memory or processing time sorting records in a cell phone.


## 6. Memory Leak

### 6.1 Definition

In computer science, a memory leak is a type of resource leak that occurs when a computer program incorrectly manages memory allocations in such a way that memory which is no longer needed is not released. 

In object-oriented programming, a memory leak may happen when an object is stored in memory but cannot be accessed by the running code. A memory leak has symptoms similar to a number of other problems and generally can only be diagnosed by a programmer with access to the program's source code.

<img src='http://www.programcreek.com/wp-content/uploads/2013/10/where-is-memory-leak-650x400.jpeg'>

### 6.2 Consequences

- A memory leak can diminish the performance of the computer by reducing the amount of available memory. All or part of the system or device stops working correctly, the application fails, or the system slows down vastly due to thrashing.

- Memory leaks may not be serious or even detectable by normal means. In modern operating systems, normal memory used by an application is released when the application terminates. This means that a memory leak in a program that only runs for a short time may not be noticed and is rarely serious.

### 6.3 Example

Pseudocode:

```cpp
while (button_pressed == true):
    floor_number* ptr = new floor_number; // which will be used to remember the floor number
    
    if (curr_floor == target_floor) // Are we already on the target floor?
        return; // if so, nothing to do
    else {
        while (lift_isIdle == false) {} // wait until the lift is idle
        goto_requiredFloor();
        delete ptr; // Release the memory we used to remember the floor number
    }
```

A simple example in C:

```c
#include <stdlib.h>

void function_which_allocates(void) {
    /* allocate an array of 45 floats */
    float *a = malloc(sizeof(float) * 45);

    /* additional code making use of 'a' */

    /* return to main, having forgotten to free the memory we malloc'd */
}

int main(void) {
    function_which_allocates();

    /* the pointer 'a' no longer exists, and therefore cannot be freed,
     but the memory is still allocated. a leak has occurred. */
}
```

## 7. Common Errors in C++

### 7.1 Programming Errors 

These are generated when typographical errors are made by users.

### 7.2 Compiler Errors 

These are errors detected by the compiler that make the program un-compilable.

Example: You forget a semi-colon (;) at the end of a statement and the compiler reports:

```cpp
int main(int argc, char const *argv[])
{
	return 0
}
```

### 7.3 Linker Errors

These are errors generated when the executable of the program cannot be generated. This may be due to wrong function prototyping, incorrect header files.

```cpp
void Foo();

int main()
{
  Foo();
  return 0;
}

void foo()
{
  // do something
}
```

### 7.4 Execution error 

These errors occur at the time of execution. Looping and arithmetic errors falls under this category.
```cpp
int scores = 500;
int num = 0;
int avg;
avg = scores / num;
```

### 7.5 Logical errors
These errors solely depend on the logical thinking of the programmer and are easy to detect if we follow the line of execution and determine why the program takes that path of execution.

```cpp
cin >> account_num;

// Assume user did not enter -1.

while (account_num != -1) {
  cout << "Account #: " << account_num << endl;
  ProcessAccount(account_num);
  // Oops...Forgot to read another account # here!
}
```

### 7.6 Segmentation Fault and Bus Errors

If the program displays these messages, it means is the program was trying to access a memory location outside its address space. The computer detects this and sends the error message back to the user. They are mainly due to out-of-bounds array references or accessing null pointer or unintialized pointers.

### 7.7 Error Detection Techniques

#### Printf /cout statements

Identify the point of the program or the sub-routine it fails, by giving a cout statement. It is always easier to debug the program once the point at which the crash occurs is known.

#### Use debugger -GDB

It allows you to see the statr of the program when it is being executed.

USAGE :
```bash
<machine>% gdb file_name.cpp core
```

## 8. Sorting algorithms

### 8.1 Definitions

Please refer to the first IQ at https://github.com/weiyialanchen/iq/blob/master/1/IQ_20150926.ipynb.

### 8.2 Example

Given [2,5,1,3,4], illustrate how to sort it?

# Algorithm Questions

### 1. Product of array except self

Discussed: https://leetcode.com/submissions/detail/38457024/

```python
class Solution(object):
    
    def cumprod(self, nums):
        results = []
        tmp = 1
        for num in nums:
            tmp *= num
            results.append(tmp)
        return results
    
    def productExceptSelf(self, nums):
        """
        :type nums: List[int]
        :rtype: List[int]
        """
        ls_left = [1] + self.cumprod(nums)[:-1]
        ls_right = self.cumprod(nums[::-1])[::-1][1:] + [1]
        return [i*j for i, j in zip(ls_left, ls_right)]
```

### 2. Find whether a linked list has cycle

A slow pointer and a fast pointer discussed: https://leetcode.com/problems/linked-list-cycle/

But we can proceed to the harder question: Given a linked list, return the node where the cycle begins. If there is no cycle, return null. (https://leetcode.com/problems/linked-list-cycle-ii/)

Path 1: $ t = X + nY + K $

Path 2: $ 2t = X + mY+ K $

Therefore, $X+K = (m-2n)Y$, walk another X steps will reach the start point.


<img src='circle.png'>

```cpp
/**
 * Definition for singly-linked list.
 * struct ListNode {
 *     int val;
 *     ListNode *next;
 *     ListNode(int x) : val(x), next(NULL) {}
 * };
 */
class Solution {
public:
    ListNode *detectCycle(ListNode *head) {
        ListNode * first = head;
        ListNode * second = head;
        
        while(first != NULL && second != NULL)
        {
            first = first->next;
            second = second->next;
            if(second != NULL)
                second = second->next;
                if(first == second)
                    break;
        }
        if(second == NULL) return NULL;
         
        // X steps
        first = head;
        while(first!=second)
        {
            first = first->next;
            second = second->next;
        }
        return second;
    }
};
```

### 3. Implement Queue using Stacks

Use another stack, we did in Leetcode, https://leetcode.com/problems/implement-queue-using-stacks/.

A similar question is https://leetcode.com/problems/implement-stack-using-queues/.

```python
class Stack(object):
    # initialize your data structure here.
    def __init__(self):
        self.queue = []

    # @param x, an integer
    # @return nothing
    def push(self, x):
        self.queue.append(x)

    # @return nothing
    def pop(self):
        for x in range(len(self.queue) - 1):
            self.queue.append(self.queue.pop(0))
        self.queue.pop(0)

    # @return an integer
    def top(self):
        top = None
        for x in range(len(self.queue)):
            top = self.queue.pop(0)
            self.queue.append(top)
        return top

    # @return an boolean
    def empty(self):
        return self.queue == []

```

### 4. Longest Sequence V

#### Question

A sequence is called an increasing sequence if each character in the sequence is strictly greater than the one before it (if there is one). For example, "abcdgz" is an increasing sequence and "abccd" and "abcda" are not increasing sequences. The comparison of the character is by the ASCII values. Similarly a sequence is called a decreasing sequence if each character in the sequence is strictly smaller than the one before it (if there is one). Now, a sequence is called a "V" sequence is if it is composed of a decreasing sequence followed by an increasing sequence with one character of overlap. For example, "zyxabcd" is a "V" sequence because it is made up of a decreasing sequence "zyxa" followed by an increasing sequence "abcd" where the two sequences overlap at the character "a". 

Your goal is to find one of the longest “V” sequences which are substrings from the input string:

#### Requirement

Your program should be able to read an input file and run as follows. The output should be written to console.
C++: ./LongestSequenceV input_file_name

#### Examples 

- Input File1: abcdabc
- Output1: abcd

(The left side is "a", the right side "abcd", and the overlap is "a". Another solution is “dabc”. Note you only need to print one of the longest v-sequence)

- Input File2: addebbbc
- Output2: eb

(The left side is “e”, and the right side is “eb”. Other solutions: “ad”, “de”,  “eb”, “bc”)

- Input File3: abababab
- Output3: bab

- Input File4: aaaa
- Output4: a

(The left side is “a”, and the right side is “a”)

#### Solution

```cpp
#include <iostream>
#include <fstream>
#include <vector>
using namespace std;
 
string longestSequenceV(string sequence) {
    string V;
    for (int i = 0; i < sequence.size(); i++) {
        int startIndex = i, endIndex = i;
        if (startIndex > 0)
            for (int j = i-1; j >= 0; j--) {
                if (sequence[j] > sequence[j+1])
                    startIndex -= 1;
                else 
                    break;
            }
        if (endIndex < sequence.size()-1)
            for (int j = i+1; j < sequence.size(); j++) {
                if (sequence[j] > sequence[j-1])
                    endIndex += 1;
                else
                    break;
            }
        string tempV = sequence.substr(startIndex, endIndex-startIndex+1);
        if (tempV.size() > V.size()) {
            V = tempV;
        }
    }
    return V;
}

int main(int argc, char const *argv[]){
    ifstream file;
    file.open(argv[1]);
    string temp;
    std::getline(file, temp);
    cout << longestSequenceV(temp) << endl;
}
```

### 5. Find Line

#### Question

Given a set of points $(X_1, Y_1), (X_2, Y_2), \cdots, (X_n, Y_n)$, find a line $Y=aX+b$, or $X=a$ that passes most points. Write a program as efficient as possible, and discuss the complexity. 

#### Requirement

Your program is required to be run as following. The output should be written to console.

C++: ./FindLine input_file_name

#### Example

- Sample file input 1:
```
4
(1,2)
(2,3)
(3,4)
(100,100)
```

- Sample output 1:
```
Y=x+1, 3
```

- Sample file input 2:
```
4
(0,1)
(0,2)
(0,3)
(100,100)
```
- Sample output 2:
```
X=0, 3
```


#### Solution

```cpp
#include <iostream>
#include <fstream>
#include <vector>
#include <map>
using namespace std;

void printOutput(vector<double> params, int freq) {
	double slope = params[0];
	double intercept = params[1];
	if (slope != numeric_limits<double>::infinity()) {
		cout << "Y=";
		if (slope == 0) {
			cout << intercept << ", " << freq << endl;
			return;
		}
		if (slope != 1)
			cout << slope;
		cout << "x+";
		if (intercept != 0)
			cout << intercept;
		cout << ", " << freq << endl;
	}
		
	else
		cout << "X=" << intercept << ", " << freq << endl;
}

void FindLine(vector<vector<int> > allPoints) {
	map<vector<double>, vector<vector<vector<int> > > > Slopes;
	for (int i = 0; i < allPoints.size(); i++) {
		for (int j = i+1; j < allPoints.size(); j++) {
			double X1 = allPoints[i][0];
			double Y1 = allPoints[i][1];
			double X2 = allPoints[j][0];
			double Y2 = allPoints[j][1];

			double slope, intercept;
			if (X1 != X2) {
				slope = (Y2 - Y1) / (X2 - X1);
				intercept = Y1 - X1 * slope;
			}
			else {
				slope = numeric_limits<double>::infinity();
				intercept = X1;
			}

			vector<double> key;
			key.push_back(slope);
			key.push_back(intercept);

			vector<vector<int> > value;
			value.push_back(allPoints[i]);
			value.push_back(allPoints[j]);

			if (Slopes.find(key) == Slopes.end()) {
				vector<vector<vector<int> > > values;
				values.push_back(value);
				Slopes[key] = values;
			}
			else 
				Slopes[key].push_back(value);
		}
	}

	int freq = 0;
	vector<double> params;
	for(map<vector<double>, vector<vector<vector<int> > > >::iterator it = Slopes.begin(); it != Slopes.end(); ++it) {
		if (it->second.size() > freq) {
			params = it->first;
			freq = it->second.size();
		}
	}

	printOutput(params, freq);
}

int main(int argc, char const *argv[]){
    ifstream file;
    file.open(argv[1]);
    string temp;
    int numberOfPoints = int(getline(file, temp));
    vector<vector<int> > allPoints;
    while (getline(file, temp)) {
    	int commaIndex = temp.find(",");
    	int X = stoi(temp.substr(1, commaIndex-1));
     	int Y = stoi(temp.substr(commaIndex+1, temp.size()-commaIndex-2));
    	vector<int> point;
    	point.push_back(X);
    	point.push_back(Y);
    	allPoints.push_back(point);
    }
    FindLine(allPoints);
}
```

### 6. Symmetric Matrix Memory

Saving and indexing.

#### 6.1 Given size n, how many space is required?

#### 6.2 Given location (x, y), where is it in linear memory?